In [1]:
# %timeit

# import pyalex

# pyalex.config.email="margree@iu.edu"

# #### warning! this takes at least 5 minutes...
# # don't run unless you need to...

# # use the OA paginate function to pull multiple pages of records
# query = pyalex.Works().filter(primary_location={"source":{"issn":"0013-936X"}}) \
#                       .paginate(per_page=200, n_max=100000)

# vol="56"
# iss="23"
# t=0
# i=0
# s=0
# for page in query:
#     v = sum([1 for p in page if 
#              (p['biblio']['volume']==vol) & 
#              (p['biblio']['issue']==iss)])
#     t += v
#     i += 1
#     s = 200*(i-1) + len(page)

# print(f"grand total: {t}")
# print(f"iterations: {i}")
# print(f"searched: {s}")

In [2]:
%timeit

import numpy as np
import pyalex

pyalex.config.email="margree@iu.edu"

#this query gets all 2013 and beyond articles from the ES&T Journal
query2 = pyalex.Works().filter(primary_location={"source":{"issn":"0013-936X"}}) \
                       .filter(publication_year='>2012') \
                       .paginate(per_page=200, n_max=None)

country_codes = []
pub_years = []
for page in query2:
    author_countries = []
    for p in page:
        if len(p['authorships'])>0: 
            author_countries = [ 
                a['institutions'][0]['country_code'] for a in p['authorships'] 
                if len(a['institutions'])>0 and ('country_code' in a['institutions'][0].keys())
            ]
            author_years = [p['publication_year']]*len(author_countries)
            
        if len(author_countries)>0:
            country_codes = np.append(country_codes, author_countries)
            pub_years = np.append(pub_years, author_years)


print(len(country_codes))
print(len(pub_years))

91593
91593


In [3]:
gsc = [
    "Afghanistan", "Algeria", "Angola", "Antigua and Barbuda", "Argentina",
    "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belize", "Benin", "Bhutan",
    "Bolivia, Plurinational State of", "Bosnia and Herzegovina", 
    "Botswana", "Brazil", "Brunei Darussalam", "Burkina Faso", "Burundi",
    "Cambodia", "Cameroon", "Cabo Verde", "Central African Republic", "Chad", 
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", #"Côte d'Ivoire", 
    "Cuba", "Korea, Democratic People's Republic of", 
    "Congo, Democratic Republic of the", "Djibouti", "Dominica", 
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", 
    "Eritrea", "Ethiopia", "Fiji", "Gabon", "Gambia", "Ghana", "Grenada", "Guatemala", 
    "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "India", "Indonesia", 
    "Iran, Islamic Republic of", "Iraq", "Jamaica", "Jordan", "Kenya", "Kiribati", 
    "Kuwait", "Lao People's Democratic Republic", 
    "Lebanon", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Malaysia", 
    "Maldives", "Mali", "Marshall Islands", "Mauritania", "Mauritius", 
    "Micronesia, Federated States of", "Mongolia", "Morocco", "Mozambique",
    "Myanmar", "Namibia", "Nauru", "Nepal", "Nicaragua", "Niger", "Nigeria", "Oman", 
    "Pakistan", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", 
    "Qatar", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", 
    "Samoa", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Seychelles", "Sierra Leone", 
    "Singapore", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Sri Lanka", 
    "Palestine, State of", "Sudan", "Suriname", "Eswatini", #"Swaziland", 
    "Syrian Arab Republic", "Tajikistan", "Thailand", "Timor-Leste", "Togo", "Tonga", 
    "Trinidad and Tobago", "Tunisia", "Turkmenistan", "Uganda", "United Arab Emirates", 
    "Tanzania, United Republic of", "Uruguay", "Vanuatu", "Venezuela, Bolivarian Republic of", 
    "Viet Nam", "Yemen", "Zambia", "Zimbabwe"
]

In [4]:
from iso3166 import countries
import numpy as np
import pandas as pd

# read-in Kaggle GDP data long format
gdp_df = pd.read_csv("../data/Kaggle_GDPbyCountry1999-2022.csv", sep=',', header=0, thousands=",")
gdp_df = pd.melt(gdp_df, id_vars=['Country'], value_vars=list(gdp_df.columns[1:]),
                 var_name='Year', value_name='GDP')
gdp_df['Year'] = gdp_df['Year'].astype(int)

country_names = {
    "Afghanistan, Rep. of.":"Afghanistan", "Bahamas, The":"Bahamas", "Bolivia":"Bolivia, Plurinational State of", 
    "Congo, Republic of":"Congo", "Congo, Democratic Republic of":"Congo, Democratic Republic of the", 
    "Cape Verde":"Cabo Verde", "Czech Republic":"Czechia", "Hong Kong SAR":"Hong Kong", 'Korea':'Korea, Republic of',
    "Kyrgyz Republic":"Kyrgyzstan", "Gambia, The":"Gambia", 'Macedonia, Former Yugoslav Republic of':"North Macedonia",
    "Moldova":"Moldova, Republic of","Russia":"Russian Federation",'São Tomé and Príncipe':'Sao Tome and Principe',
    'Slovak Republic':'Slovakia','St. Kitts and Nevis':'Saint Kitts and Nevis', 'St. Lucia':'Saint Lucia',
    'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines','Swaziland':'Eswatini','Taiwan Province of China':'Taiwan',
    'Tanzania':'Tanzania, United Republic of','Timor-Leste, Dem. Rep. of':'Timor-Leste','Turkey':'Türkiye',
    'United Kingdom':'United Kingdom of Great Britain and Northern Ireland','United States':'United States of America',
    'Venezuela':'Venezuela, Bolivarian Republic of','Vietnam':'Viet Nam', "Yemen, Republic of":"Yemen"
}
gdp_df.replace({'Country':country_names},inplace=True)

gdp_df['alpha2'] = gdp_df.apply(lambda row: countries.get(row['Country']).alpha2, axis=1)
gdp_df['GlobalSouth'] = gdp_df.apply(lambda row: 'True' if row['Country'] in gsc else 'False', axis=1)
gdp_df['GDP'] = gdp_df.apply(lambda row: 1 if row['GDP']==0 else row['GDP'], axis=1)

# country codes and author counts
pub_df = pd.DataFrame({'alpha2':country_codes, 'Year':pub_years})
pub_df.dropna(inplace=True)

pub_df['Year'] = pub_df['Year'].astype(int)

pub_df = pub_df.groupby(by=['alpha2','Year']).size().reset_index()
pub_df.columns = ['alpha2','Year','AuthorPubCount']

# merge the author count data and 
df = gdp_df.merge(pub_df, on=['alpha2', 'Year'])

df

,Country,Year,GDP,alpha2,GlobalSouth,AuthorPubCount
0,Argentina,2013,329.332,AR,True,13
1,Australia,2013,1078.910,AU,False,242
2,Austria,2013,417.771,AT,False,30
3,Bangladesh,2013,123.447,BD,True,3
4,Belgium,2013,520.026,BE,False,59
...,...,...,...,...,...,...
625,Thailand,2022,585.586,TH,True,7
626,Türkiye,2022,844.534,TR,False,2
627,Uganda,2022,47.000,UG,True,4
628,United Kingdom of Great Britain and Northern I...,2022,3442.210,GB,False,157


In [6]:
import altair as alt

alt.renderers.enable('default')  # you might need to change this....

years = df['Year'].unique()
dropdown = alt.binding_select(options=np.append(None,years), labels=np.append('All',years))
select = alt.selection_point(fields=['Year'], bind=dropdown, name='Year', value=None)

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('GDP').scale(type="log"),
    y=alt.Y('AuthorPubCount').scale(type="log"),
    color=alt.Color('GlobalSouth'),
    tooltip=['Country','GDP','AuthorPubCount','GlobalSouth','Year']
).add_params(
    select
).transform_filter(
    select
).properties(
    width=700,
    height=500
).interactive()

# chart.save('phish_songs_interactive.html')

chart


alt.Chart(...)